In [1]:
import numpy as np
import pandas as pd


from collections import OrderedDict
from datetime import *
from  math  import *
import statsmodels.api as sm
import numpy.linalg as la   #用来做线性代数运算


In [ ]:
# from WindPy import *
# # import talib as ta
# # from talib.abstract import *
# # w.start()

首先采用**2021年8月1号**的沪深300为选股的股票池，然后选用数据的时间为*2010年1月1日至2021年12月31日*的数据来建立模型，在这之前，必须去掉2010年之前未上市的股票，筛选过后共有171只股票。

In [ ]:
# codes =list(w.wset("sectorconstituent", "date=2021-08-01;windcode=000300.SH",usedf=True)[1].iloc[:,1])
# date = '2010-01-01'
# date = datetime.strptime(date,'%Y-%m-%d')
# df = w.wss(','.join(codes), "ipo_issuedate",usedf=True)[1]
# select_codes = df.loc[df['IPO_ISSUEDATE']<=date].index.tolist()
# len(select_codes)

In [2]:
#估值因子
def get_values_factor(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        
        #估值因子value_factor
        factors_codes= "pe_ttm,pe_lyr,pb_lf,pb_lyr,pcf_ncf_ttm,pcf_ocf_ttm,ps_ttm,ps_lyr,val_mvtofcff" 
        factors_names=['EP_TTM','EP_LYR','BP_LF','BP_LYR','NCF_TTM','OCF_TTM','SP_TTM','SP_LYR','FCFP_LYR'] 
        factors_value=w.wss(stocks,factors_codes,"tradeDate="+date) 
        factors_value=pd.DataFrame(factors_value.Data,index=factors_names,columns=factors_value.Codes).T
        factors_value=1/factors_value
            
  
        
        #获取企业价值倍数
        factors_value['EV/EBITDA']=w.wss(stocks, "ev2_to_ebitda","tradeDate="+date).Data[0]
        
        #获取股息率
        factors_value['DYR']=w.wss(stocks, "dividendyield2","tradeDate="+date).Data[0]
        
        dict_df[date]=factors_value
    factors_values=pd.concat(dict_df.values(),keys=dict_df.keys())
    return factors_values
#规模因子
def get_size_factor(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        size_factors=w.wss(stocks, "val_lnmv,val_lnfloatmv,val_lntotassets","tradeDate="+date)
        factors_names=['LN_MV','LN_FLOAT_MV','LN_TOTAL_ASSETS']                       
        size_factors=pd.DataFrame(size_factors.Data,index=factors_names,columns=size_factors.Codes).T
        dict_df[date]=size_factors.iloc[:,:]
        #print(dict_df.values())
        #print(dict_df.keys())
    size_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return size_factors

#杠杆因子
def get_leverage_factors(dates,stocks,factors_codes,factors_names):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        leverage_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        leverage_factors=pd.DataFrame(leverage_factors.Data,index=factors_names,columns=leverage_factors.Codes).T
        dict_df[date]=leverage_factors
    leverage_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return leverage_factors
#技术因子
def get_Technical_factors(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        factors_codes="tech_rvi,tech_rstr12,tech_cyf,tech_cry,tech_cr20"
        factors_names=['RVI','RSTR12','CYF','CRY','CR20']
        Technical_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        Technical_factors=pd.DataFrame(Technical_factors.Data,index=factors_names,columns=Technical_factors.Codes).T
         #获取RSI指标
        Technical_factors['RSI']=w.wss(stocks, "RSI","tradeDate="+date+";RSI_N=6;priceAdj=F;cycle=D").Data[0] 
        #获取DEA异同平均数指标
        Technical_factors['DEA']=w.wss(stocks, "MACD","tradeDate="+date+";MACD_L=26;MACD_S=12;MACD_N=9;MACD_IO=2;priceAdj=F;cycle=D").Data[0]
        #获取MACD指标
        Technical_factors['MACD']=w.wss(stocks, "MACD","tradeDate="+date+";MACD_L=26;MACD_S=12;MACD_N=9;MACD_IO=3;priceAdj=F;cycle=D").Data[0]
        #获取K\D\J
        Technical_factors['K']=w.wss(stocks, "KDJ","tradeDate="+date+";KDJ_N=9;KDJ_M1=3;KDJ_M2=3;KDJ_IO=1;priceAdj=F;cycle=D").Data[0]
        Technical_factors['D']=w.wss(stocks, "KDJ","tradeDate="+date+";KDJ_N=9;KDJ_M1=3;KDJ_M2=3;KDJ_IO=2;priceAdj=F;cycle=D").Data[0]
        Technical_factors['J']=w.wss(stocks, "KDJ","tradeDate="+date+";KDJ_N=9;KDJ_M1=3;KDJ_M2=3;KDJ_IO=3;priceAdj=F;cycle=D").Data[0]
        
        dict_df[date]=Technical_factors
    Liquidation_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return Liquidation_factors
#动量因子
def get_Momentum_factors(dates,stocks):
    dict_df=OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        factors_codes="tech_revs5,tech_revs10,tech_revs60,tech_revs120,tech_revs250,tech_revs750,tech_revs1mmax,tech_lnhighlow20d"
        factors_names=['REV_5D','REV_10D','REV_3M','REV_6M','REV_1Y','REV_3Y','REV_LAST1M_MAX','LN_HIGH-LOW']
        Momentum_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        Momentum_factors=pd.DataFrame(Momentum_factors.Data,index=factors_names,columns=Momentum_factors.Codes).T
        dict_df[date]=Momentum_factors
    Momentum_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return Momentum_factors

#获取成长因子
def get_growth_factors(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        factors_codes= "fa_orgr_ttm,fa_nagr,fa_gpmgr_ttm,fa_npgr_ttm,fa_tagr,fa_ncgr_ttm,fa_cfigr_ttm,fa_cffgr_ttm,fa_cfogr_ttm,fa_oigr_ttm" 
        factors_names=['sales_gr_TTM','net_asset_gr_TTM','gross_margin_gr_TTM','net_profit_gr_TTM','total_asset_gr_TTM','net_cash_flow_gr_TTM','invest_cash_flow_gr_TTM','finance_cash_folw_gr_TTM','operate_cash_flow_gr_TTM','operete_profit_gr_TTM']
        growth_factors=w.wss(stocks,factors_codes,"tradeDate="+date)
        growth_factors=pd.DataFrame(growth_factors.Data,index=factors_names,columns=growth_factors.Codes).T
        #growth_factors['eps_growth_TTM']=w.wss(A_stocks, "yoyeps_basic","rptDate="+date+";N=1").Data[0]  #基本每股收益同比增长率
        #growth_factors['roe_growth_TTM']=w.wss(A_stocks, "growth_roe","rptDate="+date+";N=1").Data[0]  #净资产收益率N年同比增长率
        dict_df[date]=growth_factors
        growth_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return growth_factors
#市值因子
def get_assisted_factors(dates,stocks):   
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        assisted_factors=w.wss(stocks, "industry_sw,mkt_cap_ashare","tradeDate="+date+';industryType=1;unit=1')
        factors_names=['INDUSTRY_SW','CAP']                       
        assisted_factors=pd.DataFrame(assisted_factors.Data,index=factors_names,columns=assisted_factors.Codes).T
        dict_df[date]=assisted_factors
    assisted_factors=pd.concat(dict_df.values(),keys=dict_df.keys())
    return assisted_factors
#获取每月交易日期序列
def get_trade_date(start_date, end_date, period='M'):
    data = w.tdays(start_date, end_date, period=period) #获取每月最后一个交易日
    trade_dates = data.Data[0]
    trade_dates = [dt.strftime("%Y-%m-%d") for dt in trade_dates]
    return trade_dates
def get_feature_names(data):  #该函数用于获取数据集中需测试的因子名
    columns = data.columns.tolist()
    fea_names = [i for i in columns if i not in ["INDUSTRY_SW",'CAP'] ]
    return fea_names
def extreme_process_MAD(Data):
    feature_names = get_feature_names(Data)
    median=Data[feature_names].median(axis=0)  #获取中位数
    MAD=abs(Data[feature_names].sub(median,axis=1)).median(axis=0)
    for j in range(len(MAD)):
        for i in range(Data.shape[0]):
            if np.isnan(Data.iloc[i,j]) == False:
                if Data.iloc[i,j] <= median[j]-5*1.4826*MAD[j]:
                    Data.iloc[i,j] = median[j]-5*1.4826*MAD[j]
                if Data.iloc[i,j] >= median[j]+5*1.4826*MAD[j]:
                    Data.iloc[i,j] = median[j]+5*1.4826*MAD[j]
                    
    return Data
        
        
def fill_missing_value(Data):
    feature_names = get_feature_names(Data)
    for j in range(len(feature_names)):
        industry_fill_value = Data[feature_names[j]].groupby(Data['INDUSTRY_SW']).mean()
        #print(j,list(industry_fill_value))
        for i in range(Data.shape[0]):
            #if i < 3:
                #print(Data.iloc[i,:]['INDUSTRY_SW'])
            if np.isnan(Data.iloc[i,j]):
                #print(industry_fill_value[Data.iloc[i,-2]])
                Data.iloc[i,j] = industry_fill_value[Data.iloc[i,-2]]
                #print(Data.iloc[i,j])
                #print(np.isnan(Data.iloc[i,j]))
                break
    return Data
def data_scale_CAP(data):
    feature_names = get_feature_names(data)
    data_=data.copy()
    cap_weight = data_["CAP"]/ data_["CAP"].sum()
    for name in feature_names:
        avg=(data_[name]*cap_weight).sum()
        data_[name]=(data_[name]-avg)/data_[name].std()
    return data_

def data_scale_neutral(data):
    feature_names = get_feature_names(data)
    data_=data.copy()
    industrys=data['INDUSTRY_SW']  #获取所属申万一级行业代码
    data_med = pd.get_dummies(data,columns=['INDUSTRY_SW'],drop_first=True)
    n = len(data['INDUSTRY_SW'].unique())    #确定产生虚拟变量个数
    X = np.array(data_med[data_med.columns[-(n-1):]])  #行业虚拟变量作为为自变量
    for name in feature_names:
        y = np.array(data_[name])
        if la.matrix_rank(X.T.dot(X)) == (n-1): #当矩阵满秩时，估计回归参数
            beta_ols = la.inv(X.T.dot(X)).dot(X.T).dot(y)  
            residual = y - X.dot(beta_ols)      #计算残差，并将其作为剔除行业影响的因子值 
        else:
            residual = y   #如果逆不存在的话 则 用原值
        data_[name]=residual
    return data_
    return data_
#因变量涨跌幅的获取以及处理
def get_pct(dates,stocks):
    dict_df = OrderedDict()
    for i in range(len(dates)-1):
        date=dates[i]
        h = "tradeDate="+date+";cycle=M"
        factors_value=w.wss(stocks,"pct_chg",h,usedf=True)[1]
        dict_df[date]=factors_value
    d=pd.concat(dict_df.values(),keys=dict_df.keys())
    return d
def accuracy(data1,data2):
    n=0
    for i in range(len(data1)):
        if data1[i] == data2[i]:
            n+=1
    acc = n/len(data1)
    return acc
def one_hot(data):
    DataY = pd.DataFrame(data)
    l = []
    for i in range(DataY.shape[1]):
        for j in range(DataY.shape[0]):
            if DataY.iloc[j,i]==0:
                l.append([1,0,0])
            elif DataY.iloc[j,i]==1:
                l.append([0,1,0])
            else:
                l.append([0,0,1])
    return l
  


In [ ]:
# start_date='20150101'
# end_date='20201231'
# dates=get_trade_date(start_date, end_date, period='M')
# values_factor=get_values_factor(dates,select_codes)
# size_factor=get_size_factor(dates,select_codes)
# factors_codes="fa_current,fa_quick,fa_blev,fa_debttoasset,fa_cfotocurliabs_ttm,fa_debttoequity"
# factors_names=['CUR','QR','BOOK_LEVEL','DEBT_TO_ASSETS','CASH_FLOW_LIABILITY','DEBT_TO_EQUITY']
# leverage_factors = get_leverage_factors(dates,select_codes,factors_codes,factors_names)
# Technical_factors = get_Technical_factors(dates,select_codes)
# Momentum_factors = get_Momentum_factors(dates,select_codes)
# assisted_factors = get_assisted_factors(dates,select_codes)
# growth_factors=get_growth_factors(dates,select_codes)
# Data= pd.concat([values_factor,growth_factors,leverage_factors,Momentum_factors,Technical_factors,assisted_factors],axis=1)


In [ ]:
# Data.to_csv('Newsvm_15_20.csv')

In [ ]:

# start_date='20200701'
# end_date='20211231'
# dates=get_trade_date(start_date, end_date, period='M')
# values_factor=get_values_factor(dates,select_codes)
# size_factor=get_size_factor(dates,select_codes)
# factors_codes="fa_current,fa_quick,fa_blev,fa_debttoasset,fa_cfotocurliabs_ttm,fa_debttoequity"
# factors_names=['CUR','QR','BOOK_LEVEL','DEBT_TO_ASSETS','CASH_FLOW_LIABILITY','DEBT_TO_EQUITY']
# leverage_factors = get_leverage_factors(dates,select_codes,factors_codes,factors_names)
# Technical_factors = get_Technical_factors(dates,select_codes)
# Momentum_factors = get_Momentum_factors(dates,select_codes)
# assisted_factors = get_assisted_factors(dates,select_codes)
# growth_factors=get_growth_factors(dates,select_codes)
# Data1= pd.concat([values_factor,growth_factors,leverage_factors,Momentum_factors,Technical_factors,assisted_factors],axis=1)


In [ ]:
# Data1.to_csv('svm_20_21.csv')

In [ ]:
# start_date='20210101'
# end_date='20211231'
# dates=get_trade_date(start_date, end_date, period='M')
# values_factor=get_values_factor(dates,select_codes)
# size_factor=get_size_factor(dates,select_codes)
# factors_codes="fa_current,fa_quick,fa_blev,fa_debttoasset,fa_cfotocurliabs_ttm,fa_debttoequity"
# factors_names=['CUR','QR','BOOK_LEVEL','DEBT_TO_ASSETS','CASH_FLOW_LIABILITY','DEBT_TO_EQUITY']
# leverage_factors = get_leverage_factors(dates,select_codes,factors_codes,factors_names)
# Technical_factors = get_Technical_factors(dates,select_codes)
# Momentum_factors = get_Momentum_factors(dates,select_codes)
# assisted_factors = get_assisted_factors(dates,select_codes)
# growth_factors=get_growth_factors(dates,select_codes)
# Data2= pd.concat([values_factor,growth_factors,leverage_factors,Momentum_factors,Technical_factors,assisted_factors],axis=1)


In [ ]:
# Data2.to_csv('svm_20_21.csv')

In [ ]:
# Data2.head()

In [ ]:
# start_date='20100101'
# end_date='20141231'
# dates=get_trade_date(start_date, end_date, period='M')
# values_factor=get_values_factor(dates,select_codes)
# size_factor=get_size_factor(dates,select_codes)
# factors_codes="fa_current,fa_quick,fa_blev,fa_debttoasset,fa_cfotocurliabs_ttm,fa_debttoequity"
# factors_names=['CUR','QR','BOOK_LEVEL','DEBT_TO_ASSETS','CASH_FLOW_LIABILITY','DEBT_TO_EQUITY']
# leverage_factors = get_leverage_factors(dates,select_codes,factors_codes,factors_names)
# Technical_factors = get_Technical_factors(dates,select_codes)
# Momentum_factors = get_Momentum_factors(dates,select_codes)
# assisted_factors = get_assisted_factors(dates,select_codes)
# growth_factors=get_growth_factors(dates,select_codes)
# Data3= pd.concat([values_factor,growth_factors,leverage_factors,Momentum_factors,Technical_factors,assisted_factors],axis=1)


In [ ]:
# Data3

In [ ]:
# Data3.to_csv('svm_10_14.csv')

拼接10-21年数据以及各自收益率

In [8]:
filepath="/Users/huyifan/Desktop/Index_Enhanccing_FYPPRO/DATA/rawdata/svm_10_14_V2.csv"
df1=pd.read_csv(filepath,index_col= [0,1])
df1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10089 entries, ('2010-01-29', '000001.SZ') to ('2014-11-28', '601998.SH')
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EP_TTM                    9752 non-null   float64
 1   EP_LYR                    9783 non-null   float64
 2   BP_LF                     9752 non-null   float64
 3   BP_LYR                    9502 non-null   float64
 4   NCF_TTM                   9783 non-null   float64
 5   OCF_TTM                   9752 non-null   float64
 6   SP_TTM                    9752 non-null   float64
 7   SP_LYR                    9752 non-null   float64
 8   FCFP_LYR                  9728 non-null   float64
 9   PEG_TTM                   9746 non-null   float64
 10  EV/EBITDA                 8485 non-null   float64
 11  DYR                       7902 non-null   float64
 12  sales_gr_TTM              9740 non-null   float64
 13  net_asset_gr_

In [14]:
filepath2="/Users/huyifan/Desktop/Index_Enhanccing_FYPPRO/DATA/rawdata/svm_15_20.csv"
df2=pd.read_csv(filepath2,index_col= [0,1])

## df2数据类型有问题，只能尝试强转
df2["sales_gr_TTM"] = pd.to_numeric(df2["sales_gr_TTM"],errors='coerce')
df2['net_asset_gr_TTM'] = pd.to_numeric(df2['net_asset_gr_TTM'],errors='coerce')
df2['gross_margin_gr_TTM'] = pd.to_numeric(df2['gross_margin_gr_TTM'],errors='coerce')
df2['net_profit_gr_TTM'] = pd.to_numeric(df2['net_profit_gr_TTM'],errors='coerce')
df2['total_asset_gr_TTM'] = pd.to_numeric(df2['total_asset_gr_TTM'],errors='coerce')
df2['net_cash_flow_gr_TTM'] = pd.to_numeric(df2['net_cash_flow_gr_TTM'],errors='coerce')
df2['invest_cash_flow_gr_TTM'] = pd.to_numeric(df2['invest_cash_flow_gr_TTM'],errors='coerce')
df2['finance_cash_folw_gr_TTM'] = pd.to_numeric(df2['finance_cash_folw_gr_TTM'],errors='coerce')
df2['operate_cash_flow_gr_TTM'] = pd.to_numeric(df2['operate_cash_flow_gr_TTM'],errors='coerce')
df2['operete_profit_gr_TTM'] = pd.to_numeric(df2['operete_profit_gr_TTM'],errors='coerce')


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12206 entries, ('2015-01-30', '000001.SZ') to ('2020-11-30', 'ErrorReport')
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EP_TTM                    12109 non-null  float64
 1   EP_LYR                    12109 non-null  float64
 2   BP_LF                     12109 non-null  float64
 3   BP_LYR                    12036 non-null  float64
 4   NCF_TTM                   12109 non-null  float64
 5   OCF_TTM                   12109 non-null  float64
 6   SP_TTM                    12109 non-null  float64
 7   SP_LYR                    12109 non-null  float64
 8   FCFP_LYR                  11990 non-null  float64
 9   PEG_TTM                   12104 non-null  float64
 10  EV/EBITDA                 10295 non-null  float64
 11  DYR                       10798 non-null  float64
 12  sales_gr_TTM              1005 non-null   float64
 13  net_asset_g

In [16]:
filepath3="/Users/huyifan/Desktop/Index_Enhanccing_FYPPRO/DATA/rawdata/svm_20_21.csv"
df3=pd.read_csv(filepath3,index_col= [0,1])
print('----------------***************')
df3.info()

----------------***************
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1881 entries, ('2021-01-29', '000001.SZ') to ('2021-11-30', '601998.SH')
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EP_TTM                    1881 non-null   float64
 1   EP_LYR                    1881 non-null   float64
 2   BP_LF                     1881 non-null   float64
 3   BP_LYR                    1870 non-null   float64
 4   NCF_TTM                   1881 non-null   float64
 5   OCF_TTM                   1881 non-null   float64
 6   SP_TTM                    1881 non-null   float64
 7   SP_LYR                    1881 non-null   float64
 8   FCFP_LYR                  1771 non-null   float64
 9   PEG_TTM                   1881 non-null   float64
 10  EV/EBITDA                 1595 non-null   float64
 11  DYR                       1695 non-null   float64
 12  sales_gr_TTM              1881 non-nu

In [17]:
tepresult = df1.append(df2)

In [18]:
tepresult1 = tepresult.append(df3)
tepresult1.head(100)
tepresult2 = tepresult1.drop(columns='INDUSTRY_SW')

In [19]:
tepresult1.to_csv('DATA/FypData.csv')
tepresult2.to_csv('DATA/FypData_Noind.csv')

In [20]:
Finaldata =  pd.read_csv('DATA/FypData.csv',index_col=[0,1])

Finaldata

EP_TTM    EP_LYR     BP_LF    BP_LYR   NCF_TTM  \
2010-01-29 000001.SZ  0.013867  0.009112  0.283262  0.303758 -0.064149   
           000002.SZ  0.045905  0.039273  0.340257       NaN  0.074708   
           000063.SZ  0.024815  0.020230  0.187205  0.205019  0.035129   
           000066.SZ  0.018792  0.001165  0.190177  0.215592 -0.002868   
           000069.SZ  0.020689  0.019296  0.278921  0.220411  0.001522   
...                        ...       ...       ...       ...       ...   
2021-11-30 601919.SH  0.264802  0.035688  0.402580  0.157870  0.357188   
           601939.SH  0.206499  0.188221  1.682237  1.572736  0.147453   
           601988.SH  0.234643  0.214805  2.072642  1.961201 -0.100108   
           601989.SH -0.003299 -0.005061  0.897110  0.896090 -0.026970   
           601998.SH  0.244398  0.222427  2.250961  2.132654 -0.059966   

                       OCF_TTM    SP_TTM    SP_LYR  FCFP_LYR   PEG_TTM  ...  \
2010-01-29 000001.SZ -0.229431  0.220895  0.215367 -0.317199 -0.934475  ...   
           000002.SZ  0.128499  0.467341  0.399159 -0.013718 -6.876091  ...   
           000063.SZ  0.056341  0.692237  0.539725  0.073643  1.193498  ...   
           000066.SZ  0.000856  0.408216  0.389139 -0.272620  0.075720  ...   
           000069.SZ  0.055874  0.089973  0.073419  0.003596  3.327908  ...   
...                        ...       ...       ...       ...       ...  ...   
2021-11-30 601919.SH  0.489749  1.024693  0.615685  0.104475  0.006364  ...   
           601939.SH  0.307786  0.561661  0.524878  0.041629  0.243087  ...   
           601988.SH  0.524404  0.660052  0.629849  0.406602  0.228316  ...   
           601989.SH  0.054446  0.367919  0.367107  0.065137 -5.260863  ...   
           601998.SH  0.381133  0.911666  0.884310  0.151245  0.188721  ...   

                               CRY      CR20        RSI       DEA      MACD  \
2010-01-29 000001.SZ  7.330568e+24  0.601909  40.679102 -0.154855  0.027383   
           000002.SZ  7.632250e+24  0.384014  28.037944 -0.261729 -0.036921   
           000063.SZ  7.422045e+24  0.861125  32.976178  0.315580 -0.499569   
           000066.SZ  7.739198e+24  1.650590  58.051269  0.292211 -0.080614   
           000069.SZ  7.806521e+22  0.644256  50.207083 -0.276231  0.094665   
...                            ...       ...        ...       ...       ...   
2021-11-30 601919.SH  7.930775e+24  1.320055  82.216088 -0.485934  0.570153   
           601939.SH  7.772932e+24  0.394595  19.826828 -0.044506 -0.003089   
           601988.SH  7.266895e+24  0.854545  32.886607  0.002518 -0.000220   
           601989.SH  7.997289e+24  0.666667  49.509567 -0.032591  0.005630   
           601998.SH  7.605546e+24  0.941176  44.700713 -0.020923  0.008855   

                              K          D          J  INDUSTRY_SW  \
2010-01-29 000001.SZ  37.374130  44.146017  23.830357           银行   
           000002.SZ  14.844958  13.735378  17.064119          房地产   
           000063.SZ  17.045961  26.749536  -2.361188           通信   
           000066.SZ  55.216970  51.375066  62.900779          计算机   
           000069.SZ  52.090819  43.111073  70.050312          房地产   
...                         ...        ...        ...          ...   
2021-11-30 601919.SH  71.285112  59.184271  95.486793         交通运输   
           601939.SH  21.008514  26.552141   9.921262           银行   
           601988.SH  53.111550  70.050291  19.234069           银行   
           601989.SH  32.366961  31.493178  34.114526         国防军工   
           601998.SH  48.278461  48.059958  48.715469           银行   

                               CAP  
2010-01-29 000001.SZ  6.345328e+10  
           000002.SZ  9.018793e+10  
           000063.SZ  6.372514e+10  
           000066.SZ  1.014895e+10  
           000069.SZ  1.971988e+10  
...                            ...  
2021-11-30 601919.SH  1.968235e+11  
           601939.SH  5.525947e+10  
           601988.SH  6.428348e+11  
           601989.SH  9.20

In [21]:
Finaldata2 =Finaldata.groupby(level=0).apply(extreme_process_MAD)

In [26]:
Finaldata2.to_csv('DATA/FypData_2.csv')
Finaldata2 = pd.read_csv('DATA/FypData_2.csv',index_col=[0,1])
Finaldata2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24176 entries, ('2010-01-29', '000001.SZ') to ('2021-11-30', '601998.SH')
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EP_TTM                    23742 non-null  float64
 1   EP_LYR                    23773 non-null  float64
 2   BP_LF                     23742 non-null  float64
 3   BP_LYR                    23408 non-null  float64
 4   NCF_TTM                   23773 non-null  float64
 5   OCF_TTM                   23742 non-null  float64
 6   SP_TTM                    23742 non-null  float64
 7   SP_LYR                    23742 non-null  float64
 8   FCFP_LYR                  23489 non-null  float64
 9   PEG_TTM                   23731 non-null  float64
 10  EV/EBITDA                 20375 non-null  float64
 11  DYR                       20395 non-null  float64
 12  sales_gr_TTM              12626 non-null  float64
 13  net_asset_gr_

In [78]:
Data3=fill_missing_value(Finaldata2)
Data3.to_csv('filldata.csv')
Data4 = pd.read_csv('filldata.csv')

In [65]:
industry_fill_value = Finaldata['CAP'].groupby(Finaldata['INDUSTRY_SW']).mean()
industry_fill_value['房地产']

57302601265.30186

In [80]:
list1 = [11665,
11693,
11774
,11837
,11946
,11493
,11521
,11602
,11321
,11349
,11430
]
s3 = []
for i in dele_list:
    if i not in list1:
        s3.append(i-1)


[11285,
 11457,
 11629,
 11801,
 11973,
 12145,
 12317,
 12489,
 12661,
 12833,
 13005,
 13177,
 13349,
 13521,
 13693,
 13865,
 14037,
 14209,
 14381,
 14553,
 14725,
 14897,
 15069,
 15241,
 15413,
 15585,
 15757,
 15929,
 16101,
 16273,
 16445,
 16617,
 16789,
 16961,
 17133,
 17305,
 17477,
 17649,
 17821,
 17993,
 18165,
 18337,
 18509,
 18681,
 18853,
 19025,
 19197,
 19369,
 19541,
 19713,
 19885,
 20057,
 20229,
 20401,
 20573,
 20745,
 20917,
 21089,
 21261,
 21433,
 21605,
 21777,
 21949,
 22121,
 22293]

In [82]:
Data4.drop(s3)
Data4.to_csv('de_error_data.csv')


,,Unnamed: 1,EP_TTM,EP_LYR,BP_LF,BP_LYR,NCF_TTM,OCF_TTM,SP_TTM,SP_LYR,FCFP_LYR,...,CRY,CR20,RSI,DEA,MACD,K,D,J,INDUSTRY_SW,CAP
,Unnamed: 0.1,,,,,,,,,,,,,,,,,,,,,
0,2010-01-29,000001.SZ,0.013867,0.009112,0.283262,0.303758,-0.064149,-0.229431,0.220895,0.215367,-0.317199,...,7.330568e+24,0.601909,40.679102,-0.154855,0.027383,37.374130,44.146017,23.830357,银行,6.345328e+10
1,2010-01-29,000002.SZ,0.045905,0.039273,0.340257,0.572917,0.074708,0.128499,0.467341,0.399159,-0.013718,...,7.632250e+24,0.384014,28.037944,-0.261729,-0.036921,14.844958,13.735378,17.064119,房地产,9.018793e+10
2,2010-01-29,000063.SZ,0.024815,0.020230,0.187205,0.205019,0.035129,0.056341,0.692237,0.539725,0.073643,...,7.422045e+24,0.861125,32.976178,0.315580,-0.499569,17.045961,26.749536,-2.361188,通信,6.372514e+10
3,2010-01-29,000066.SZ,0.018792,0.001165,0.190177,0.215592,-0.002868,0.000856,0.408216,0.389139,-0.272620,...,7.739198e+24,1.650590,58.051269,0.292211,-0.080614,55.216970,51.375066,62.900779,计算机,1.014895e+10
4,2010-01-29,000069.SZ,0.020689,0.019296,0.278921,0.220411,0.001522,0.055874,0.089973,0.073419,0.003596,...,7.806521e+22,0.644256,50.207083,-0.276231,0.094665,52.090819,43.111073,70.050312,房地产,1.971988e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24171,2021-11-30,601919.SH,0.264802,0.035688,0.402580,0.157870,0.357188,0.489749,1.024693,0.615685,0.104475,...,7.930775e+24,1.320055,82.216088,-0.485934,0.570153,71.285112,59.184271,95.486793,交通运输,1.968235e+11
24172,2021-11-30,601939.SH,0.206499,0.188221,1.682237,1.572736,0.147453,0.307786,0.561661,0.524878,0.041629,...,7.772932e+24,0.394595,19.826828,-0.044506,-0.003089,21.008514,26.552141,9.921262,银行,5.525947e+10
24173,2021-11-30,601988.SH,0.234643,0.214805,2.072642,1.961201,-0.100108,0.524404,0.660052,0.629849,0.406602,...,7.266895e+24,0.854545,32.886607,0.002518,-0.000220,53.111550,70.050291,19.234069,银行,6.428348e+11


In [91]:
newdata5 = pd.read_csv('/Users/huyifan/Desktop/Index_Enhanccing_FYPPRO/DATA/de_error_data.csv',index_col=[0,1])
newdata5.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24111 entries, ('2010-01-29', '000001.SZ') to ('2021-11-30', '601998.SH')
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EP_TTM                    23749 non-null  float64
 1   EP_LYR                    23780 non-null  float64
 2   BP_LF                     23749 non-null  float64
 3   BP_LYR                    23415 non-null  float64
 4   NCF_TTM                   23780 non-null  float64
 5   OCF_TTM                   23749 non-null  float64
 6   SP_TTM                    23749 non-null  float64
 7   SP_LYR                    23749 non-null  float64
 8   FCFP_LYR                  23496 non-null  float64
 9   PEG_TTM                   23738 non-null  float64
 10  EV/EBITDA                 20375 non-null  float64
 11  DYR                       20402 non-null  float64
 12  sales_gr_TTM              12633 non-null  float64
 13  net_asset_gr_

In [92]:
for i in range(newdata5.shape[0]):
    if np.isnan(newdata5.iloc[i,-1]):
          newdata5.iloc[i,-1] = industry_fill_value[newdata5.iloc[i,:]['INDUSTRY_SW']]

In [93]:
# ## 流程 行业填充缺失
# dele_list = []
# for i in range(newdata5.shape[0]): ### row 数
#     if np.isnan(newdata5.iloc[i,-1]):
#         print(newdata5.iloc[i,:]['INDUSTRY_SW'])
#         print(i)
        
        
#         #Data3.iloc[i,-1] = industry_fill_value[]
        
        
          


In [94]:



newdata5 = newdata5.fillna(newdata5.mean())
newdata6 = data_scale_CAP(newdata5)
newdata7 = data_scale_neutral(newdata6)
newdata8 = newdata7.drop(['INDUSTRY_SW'],axis=1)
mean = newdata8.mean()
std = newdata8.std()
newdata9 = (newdata8-mean)/std
newdata9.to_csv('DATA/FYP_X.csv')


In [101]:
## 获取因变量pct 涨幅
pct= pd.read_csv('/Users/huyifan/Desktop/Index_Enhanccing_FYPPRO/DATA/rawdata/pct_10_21.csv',index_col=[0,1])
y = pct.dropna()



In [103]:
train_y = pd.concat([y.head(2000),y.tail(2000)])
train_y2 = train_y.reset_index()
train_y2

,level_0,level_1,PCT_CHG
0,2010-02-26,000001.SZ,3.456221
1,2010-02-26,000002.SZ,0.963597
2,2010-02-26,000063.SZ,6.239926
3,2010-02-26,000066.SZ,-2.222222
4,2010-02-26,000069.SZ,0.326797
...,...,...,...
3995,2021-11-30,601919.SH,5.081670
3996,2021-11-30,601939.SH,-2.866779
3997,2021-11-30,601988.SH,0.000000
3998,2021-11-30,601989.SH,0.724638


In [ ]:
# 划分训练&测试
pct_dates.insert(0,dates[0])#为了让日期索引对齐加入第一个月份
#将每个月份前退一个月的交易日
for i in range(len(train_y2.iloc[:,0])):
    train_y2.iloc[i,0]=pct_dates[pct_dates.index(train_y2.iloc[i,0])-1]
train_y3 = train_y2.set_index(['level_0','level_1'])
Y = train_y3#采用的是月的涨跌幅所以会比较大

In [ ]:
X = pd.read_csv('data/SVM_X.csv',index_col=[0,1])
X = X.loc[Y.index]
Y.iloc[:2000,0]=0
Y.iloc[2000:,0]=1

In [ ]:
start_date='20100101'
end_date='20211230'
dates=get_trade_date(start_date, end_date, period='M')
dates